In [ ]:
import torch
import torch.nn as nn
import numpy
import pathlib
from data_utils import MRAData
from trainer import Solver
from blocks import *
import os
import nibabel as nib

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings(action='ignore')

torch.cuda.is_available()

In [ ]:
dataset_path = pathlib.Path("/home/dhaval/adam_data")

size = [20, 20, 20]

train_data = MRAData(dataset_path, patch_size=size, mode="train", transform="normalize", leave_num=0)
val_data = MRAData(dataset_path, patch_size=size, mode="val", transform="normalize", leave_num=0)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=2, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=2, shuffle=False, num_workers=2)

In [ ]:
model = UNet(in_channels=1, out_channels=2, final_activation="")

optim_args_SGD = {"lr": 3e-2, "weight_decay": 0.005, "momentum": 0.9, "nesterov": True}
optim_args_Adam = {"lr": 5e-2, "weight_decay": 0.005}

solver = Solver(optim_args=optim_args_SGD, optim=torch.optim.SGD)
solver.train(model, train_loader, val_loader, log_nth=5, num_epochs=20)

In [ ]:
torch.save(model.state_dict(), "unet.pth")